In [ ]:
# This file is part of the Minnesota Population Center's NHGISXWALK.
# For copyright and licensing information, see the NOTICE and LICENSE files
# in this project's top-level directory, and also on-line at:
#   https://github.com/ipums/nhgisxwalk

# Generating Subsets for testing: 2000
##  blocks, block groups parts, and blocks


1. From a national crosswalk: 
 1. Create target state-level subsets for NHGIS base crosswalks
 1. Create target state-level subsets for NHGIS base tabular data
 1. Record unit tests values for posterity



**This is currently only intended for use with block-level data as base units.**


**James Gaboardi** **(<jgaboardi@gmail.com>), 2020-05**

In [1]:
%load_ext watermark
%watermark

2020-10-01T17:36:58-04:00

CPython 3.8.5
IPython 7.18.1

compiler   : Clang 10.0.1 
system     : Darwin
release    : 19.6.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [2]:
import inspect
import nhgisxwalk
import numpy
import pandas

%load_ext autoreload
%autoreload 2
%watermark -w
%watermark -iv

watermark 2.0.2
numpy      1.19.1
pandas     1.1.1
nhgisxwalk 0.0.9post1



### Set the state (for subsetting), source & target, and year & geography

In [3]:
subset_state = "10" # Delaware
source_year, target_year = "2000", "2010"
gj_src, gj_trg = "GJOIN%s"%source_year, "GJOIN%s"%target_year

In [4]:
# Set these to a local directory
data_in = "path/to/data/"
data_tab = "path/to/data/"

In [5]:
data_out = "../testing_data_subsets/"
block_file = "%s_block" % source_year

### Set the base-level crosswalk file name

In [6]:
base_xwalk_name = "nhgis_blk%s_blk%s_gj" % (source_year, target_year)
base_xwalk_name

'nhgis_blk2000_blk2010_gj'

### Set the base (source) summary file name

In [7]:
base_source_name = "%s/%s.csv" % (block_file, block_file)
base_source_file = "%s%s" % (data_tab, base_source_name)

### Read in the national the base-level crosswalk

In [8]:
data_types = nhgisxwalk.str_types([gj_src, gj_trg])
from_csv_kws = {"path": data_in, "archived": True, "remove_unpacked": True}
read_csv_kws = {"dtype": data_types}
base_xwalk = nhgisxwalk.xwalk_df_from_csv(
    base_xwalk_name, **from_csv_kws, **read_csv_kws
)
base_xwalk.head()

,GJOIN2000,GJOIN2010,WEIGHT,PAREA
0,G01000100201001000,G01000100201002000,0.035897,0.008988
1,G01000100201001000,G01000100201002001,0.253330,0.263725
2,G01000100201001000,G01000100201002002,0.000000,0.000385
3,G01000100201001000,G01000100201002003,0.076297,0.055430
4,G01000100201001000,G01000100201002004,0.032441,0.007543


### Create the state subset of the base-level crosswalk (for use in GH testing)

In [9]:
lambda_func = lambda x: x[1:3] == subset_state
ss_base = base_xwalk[base_xwalk["GJOIN2010"].map(lambda_func)].copy()
ss_base.reset_index(drop=True, inplace=True)

### Declare input variable
**not needed for creating a subset perse, but should do regardless**

In [10]:
input_vars = [
    nhgisxwalk.desc_code_2000_SF1b["Persons"]["Total"],
    nhgisxwalk.desc_code_2000_SF1b["Families"]["Total"],
    nhgisxwalk.desc_code_2000_SF1b["Households"]["Total"],
    nhgisxwalk.desc_code_2000_SF1b["Housing Units"]["Total"]
]
input_var_tags = ["pop", "fam", "hh", "hu"]

### Generate the desired crosswalk and subset down to the target state

In [11]:
state_bgp2000tr2010 = nhgisxwalk.GeoCrossWalk(
    base_xwalk,
    source_year=source_year,
    target_year=target_year,
    source_geo="bgp",
    target_geo="tr",
    base_source_table=base_source_file,
    input_var=input_vars,
    weight_var=input_var_tags,
    keep_base=True,
    add_geoid=True,
    stfips=subset_state
)
del base_xwalk
state_bgp2000tr2010.xwalk

,bgp2000gj,tr2010gj,tr2010ge,wt_pop,wt_fam,wt_hh,wt_hu
0,G10000109044444430042202U1,G1000010042202,10001042202,1.0,1.0,1.0,1.0
1,G10000109044461265042201R1,G1000010042201,10001042201,1.0,1.0,1.0,1.0
2,G10000109044461265042201U1,G1000010042201,10001042201,1.0,1.0,1.0,1.0
3,G10000109044461265042201U2,G1000010042201,10001042201,1.0,1.0,1.0,1.0
4,G10000109044461480042202R2,G1000010042202,10001042202,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
1038,G10000509355299999051500R4,G1000050051500,10005051500,1.0,1.0,1.0,1.0
1039,G10000509355299999051500U1,G1000050051500,10005051500,1.0,1.0,1.0,1.0
1040,G10000509355299999051500U3,G1000050051500,10005051500,1.0,1.0,1.0,1.0
1041,G10000509355299999051500U4,G1000050051500,10005051500,1.0,1.0,1.0,1.0


In [12]:
state_bgp2000tr2010.xwalk.drop_duplicates(subset=["bgp2000gj", "tr2010gj"])

,bgp2000gj,tr2010gj,tr2010ge,wt_pop,wt_fam,wt_hh,wt_hu
0,G10000109044444430042202U1,G1000010042202,10001042202,1.0,1.0,1.0,1.0
1,G10000109044461265042201R1,G1000010042201,10001042201,1.0,1.0,1.0,1.0
2,G10000109044461265042201U1,G1000010042201,10001042201,1.0,1.0,1.0,1.0
3,G10000109044461265042201U2,G1000010042201,10001042201,1.0,1.0,1.0,1.0
4,G10000109044461480042202R2,G1000010042202,10001042202,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
1038,G10000509355299999051500R4,G1000050051500,10005051500,1.0,1.0,1.0,1.0
1039,G10000509355299999051500U1,G1000050051500,10005051500,1.0,1.0,1.0,1.0
1040,G10000509355299999051500U3,G1000050051500,10005051500,1.0,1.0,1.0,1.0
1041,G10000509355299999051500U4,G1000050051500,10005051500,1.0,1.0,1.0,1.0


In [13]:
state_bgp2000tr2010.xwalk["bgp2000gj"].nunique()

908

In [14]:
state_bgp2000tr2010.xwalk["tr2010gj"].nunique()

218

### Write out the state subset of the base-level crosswalk (for use in GH testing)

In [15]:
out_path = "%s%s" % (data_out, base_xwalk_name)
nhgisxwalk.prepare_data_product(ss_base, base_xwalk_name, out_path, remove=True)

In [16]:
ss_base

,GJOIN2000,GJOIN2010,WEIGHT,PAREA
0,G10000100401001000,G10000100401001000,1.000000,1.000000
1,G10000100401001001,G10000100401001001,0.999981,0.999988
2,G10000100401001001,G10000100401001003,0.000019,0.000012
3,G10000100401001002,G10000100401001002,1.000000,1.000000
4,G10000100401001003,G10000100401001003,1.000000,1.000000
...,...,...,...,...
28471,G10000500519002095,G10000500519002075,1.000000,1.000000
28472,G10000500519002096,G10000500519002131,1.000000,1.000000
28473,G10000500519002097,G10000500519002130,1.000000,1.000000
28474,G10000500519002098,G10000500519002079,1.000000,1.000000


### Record, subset, and write out the 1990 BLKs (sf1) needed to create this subset

In [17]:
blk2000 = ss_base[~ss_base["GJOIN2000"].isna()]["GJOIN2000"]
blk2000

0        G10000100401001000
1        G10000100401001001
2        G10000100401001001
3        G10000100401001002
4        G10000100401001003
                ...        
28471    G10000500519002095
28472    G10000500519002096
28473    G10000500519002097
28474    G10000500519002098
28475    G34003300204002001
Name: GJOIN2000, Length: 28476, dtype: object

In [18]:
blk2000.shape

(28476,)

In [19]:
blk2000.nunique()

17484

In [20]:
# read in base source file
base_source_df = pandas.read_csv(base_source_file, dtype=str)
base_source_df

,GISJOIN,YEAR,STATE,STATEA,COUNTY,COUNTYA,CTY_SUBA,PLACEA,TRACTA,BLCK_GRPA,BLOCKA,AIANHHA,URBRURALA,NAME,FXS001,FY4001,F2V001,FV5001
0,G01000100201001000,2000,Alabama,01,Autauga,001,92628,99999,020100,1,1000,9999,R,Block 1000,16,7,4,7
1,G01000100201001001,2000,Alabama,01,Autauga,001,92628,62328,020100,1,1001,9999,R,Block 1001,40,13,13,14
2,G01000100201001002,2000,Alabama,01,Autauga,001,92628,62328,020100,1,1002,9999,U,Block 1002,284,101,88,104
3,G01000100201001003,2000,Alabama,01,Autauga,001,92628,62328,020100,1,1003,9999,U,Block 1003,47,16,13,17
4,G01000100201001004,2000,Alabama,01,Autauga,001,92628,62328,020100,1,1004,9999,U,Block 1004,18,7,4,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8205577,G56004509513003080,2000,Wyoming,56,Weston,045,92255,99999,951300,3,3080,9999,R,Block 3080,0,0,0,0
8205578,G56004509513003081,2000,Wyoming,56,Weston,045,92255,99999,951300,3,3081,9999,R,Block 3081,0,0,0,0
8205579,G56004509513003082,2000,Wyoming,56,Weston,045,92255,99999,951300,3,3082,9999,R,Block 3082,0,0,0,0
8205580,G56004509513003083,2000,Wyoming,56,Weston,045,92255,99999,951300,3,3083,9999,R,Block 3083,1,1,0,1


In [21]:
uniques = base_source_df["GISJOIN"].isin(blk2000.unique())
base_source_df = base_source_df[uniques]
base_source_df.reset_index(drop=True, inplace=True)
base_source_df

,GISJOIN,YEAR,STATE,STATEA,COUNTY,COUNTYA,CTY_SUBA,PLACEA,TRACTA,BLCK_GRPA,BLOCKA,AIANHHA,URBRURALA,NAME,FXS001,FY4001,F2V001,FV5001
0,G10000100401001000,2000,Delaware,10,Kent,001,91480,99999,040100,1,1000,9999,R,Block 1000,15,9,6,9
1,G10000100401001001,2000,Delaware,10,Kent,001,91480,99999,040100,1,1001,9999,R,Block 1001,364,104,94,104
2,G10000100401001002,2000,Delaware,10,Kent,001,91480,99999,040100,1,1002,9999,R,Block 1002,16,6,6,6
3,G10000100401001003,2000,Delaware,10,Kent,001,91480,99999,040100,1,1003,9999,R,Block 1003,80,27,24,27
4,G10000100401001004,2000,Delaware,10,Kent,001,91480,99999,040100,1,1004,9999,R,Block 1004,55,20,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17479,G10000500519002095,2000,Delaware,10,Sussex,005,91628,20380,051900,2,2095,9999,U,Block 2095,18,9,4,12
17480,G10000500519002096,2000,Delaware,10,Sussex,005,91628,20380,051900,2,2096,9999,R,Block 2096,3,2,1,2
17481,G10000500519002097,2000,Delaware,10,Sussex,005,91628,99999,051900,2,2097,9999,U,Block 2097,0,0,0,0
17482,G10000500519002098,2000,Delaware,10,Sussex,005,91628,99999,051900,2,2098,9999,U,Block 2098,18,5,3,5


In [22]:
base_source_df.to_csv("%s%s.csv.zip" % (data_out, block_file))

-----------------